In [171]:
""" FINE TUNING MODEL WITHOUT FAIRNESS OR DIFFERENTIAL PRIVACY """
import heapq
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from models import NCF
from fairness_measures import Measures

import data
from importlib import reload
reload(data)
from data import AttributeData, TargetData




In [172]:
emb_size = 128
hidden_layers = np.array([emb_size, 64, 32, 16])
output_size = 1

num_epochs = 10
batch_size = 256

num_negatives = 5

random_samples = 15
top_k = 10

learning_rate = .001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fairness_thres = torch.tensor(0.1).to(device)
epsilonBase = torch.tensor(0.0).to(device)


In [173]:
# targets= TargetData()
data = AttributeData()
m = Measures()


/Users/franciscocancedda/Documents/Scool/NFPCF/data.py:221: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/franciscocancedda/Documents/Scool/NFPCF/data.py:222: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_id = clean[['ojob']].drop_duplicates()


In [205]:
# LOAD PRE-TRAINED MODEL
ncf = NCF(6040, 3952, emb_size, hidden_layers, output_size).to(device)
ncf.load_state_dict(torch.load("models/preTrained_NCF"))

# FETCH NUMBER OF UNIQUE CAREERS
n_careers = data.num_jobs

# CHANGE EMBEDDING SIZE TO FIT SENSITIVE INFO
# ncf.like_emb = nn.Embedding(n_careers, emb_size).to(device)


In [178]:
user_embeds = ncf.user_emb.weight.data.cpu().detach().numpy()
user_embeds = user_embeds.astype('float')


''' COMPUTE GENDER EMBEDDING '''
gender_embed = np.zeros((2,user_embeds.shape[1]))
num_users_x_group = np.zeros((2, 1))

for i in range(data.train.shape[0]):
    u = data.train['uid'].iloc[i]
    if data.train['gender'].iloc[i] == 0:
        gender_embed[0] +=  user_embeds[u]
        num_users_x_group[0] += 1.0
    else:
        gender_embed[1] +=  user_embeds[u]
        gender_embed[1] += 1.0
        num_users_x_group[1] += 1.0


In [179]:
''' VERTICAL BIAS'''
gender_embed = gender_embed / num_users_x_group
# vBias = compute_bias_direction(gender_embed)
vBias = gender_embed[1].reshape((1,-1)) - gender_embed[0].reshape((1,-1))
vBias = vBias / np.linalg.norm(vBias,axis=1,keepdims=1)

vBias

array([[0.08833937, 0.09018802, 0.0879173 , 0.09250274, 0.08757752,
        0.08920489, 0.08567962, 0.08674757, 0.08743061, 0.09037649,
        0.08797264, 0.09147833, 0.09224037, 0.09063672, 0.09083148,
        0.0854365 , 0.08963342, 0.0895668 , 0.08999261, 0.08771025,
        0.08754062, 0.09176309, 0.0865631 , 0.09061284, 0.0840617 ,
        0.08517442, 0.08619777, 0.08566763, 0.09502302, 0.09061725,
        0.08765419, 0.08803402, 0.08960038, 0.08851412, 0.08357468,
        0.0895203 , 0.09011151, 0.08341624, 0.08742171, 0.09232834,
        0.0885374 , 0.08852077, 0.08717989, 0.08961292, 0.08994275,
        0.09027861, 0.09130058, 0.0902064 , 0.08928831, 0.08815647,
        0.09112686, 0.08360929, 0.09076972, 0.08280671, 0.09078205,
        0.08712893, 0.08788025, 0.08945807, 0.0916801 , 0.09049211,
        0.08781245, 0.08455281, 0.08489517, 0.08846074, 0.08586793,
        0.08481279, 0.08591463, 0.08852091, 0.0919617 , 0.09057988,
        0.08801147, 0.09121689, 0.08738795, 0.08

In [180]:
''' LINEAR PROJECTION '''
debiased_user_embeds = user_embeds
for i in range(len(data.df)):
    u = data.df['uid'].iloc[i]
    debiased_user_embeds[u] = user_embeds[u] - (np.inner(user_embeds[u].reshape(1,-1),vBias)[0][0])*vBias



In [181]:
'''UPDATE USER EMBEDDINGS'''
fairness_thres = torch.tensor(0.1).to(device)
epsilonBase = torch.tensor(0.0).to(device)

n_careers = data.num_jobs

# replace page items with career items
ncf.like_emb = nn.Embedding(n_careers,emb_size).to(device)
# freeze user embedding
ncf.user_emb.weight.requires_grad=False

# replace user embedding of the model with debiased embeddings
ncf.user_emb.weight.data = torch.from_numpy(debiased_user_embeds.astype(np.float32)).to(device)



In [182]:
# criterion = nn.BCELoss()

# optimizer = torch.optim.Adam(ncf.parameters(), lr=learning_rate, weight_decay=1e-6)

ncf.train()


NCF(
  (user_emb): Embedding(6040, 128)
  (like_emb): Embedding(17, 128)
  (fc1): Linear(in_features=256, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=32, bias=True)
  (relu3): ReLU()
  (fc4): Linear(in_features=32, out_features=16, bias=True)
  (relu4): ReLU()
  (outLayer): Linear(in_features=16, out_features=1, bias=True)
  (out_act): Sigmoid()
)

In [183]:
def evaluate_fine_tune(model, df_val, k, random_samples):
    model.eval()
    avg_hr = np.zeros((len(df_val), k))
    avg_ndcg = np.zeros((len(df_val), k))

    for i in range(len(df_val)):
        test_df = data.add_negatives(
            df_val,
            item='job',
            items=data.jobs,
            n_samples=random_samples
        )
        users, items = torch.LongTensor(test_df.uid).to(device), torch.LongTensor(test_df.job).to(device)
        y_hat = model(users, items)

        y_hat = y_hat.cpu().detach().numpy().reshape((-1,))
        items = items.cpu().detach().numpy().reshape((-1,))
        map_item_score = {}
        for j in range(len(y_hat)):
            map_item_score[items[j]] = y_hat[j]
        for k in range(k):
            # Evaluate top rank list
            ranklist = heapq.nlargest(k, map_item_score, key=map_item_score.get)
            gtItem = items[0]
            avg_hr[i, k] = m.get_hit_ratio(ranklist, gtItem)
            avg_ndcg[i, k] = m.get_ndcg(ranklist, gtItem)
        avg_hr = np.mean(avg_hr, axis=0)
        avg_ndcg = np.mean(avg_ndcg, axis=0)
        return avg_hr, avg_ndcg

In [184]:
# LOAD TRAINING DATA
all_users = torch.LongTensor(data.train['uid'].values).to(device)
all_items = torch.LongTensor(data.train['job'].values).to(device)

# PROTECTED ATTRIBUTE
all_genders = torch.LongTensor(data.train['gender'].values).to(device)
# from opacus import PrivacyEngine
#
# privacy_engine = PrivacyEngine(
#     ncf,
#     sample_rate=0.01,
#     alphas=[10, 100],
#     noise_multiplier=1.3,
#     max_grad_norm=1.0,
# )
# optimizer = torch.optim.Adam(ncf.parameters(), lr=learning_rate, weight_decay=1e-6)
# privacy_engine.attach(optimizer)

In [203]:
def train_normal():
    # REMOVES JOBS BASED ON THRESHOLD + SPLIT DATA
    # train, test = data.train_test_split(train_fraction)
    # num_batches = np.int64(np.floor(train.shape[0] / batch_size))
    loss = nn.BCELoss()
    optimizer = torch.optim.SGD(ncf.parameters(), lr=learning_rate, weight_decay=1e-6)
    final_loss = 0

    for i in range(num_epochs):
        j=0
        dataloader = DataLoader(data, batch_size=batch_size,
                                shuffle=True, num_workers=0)

        it_per_epoch = len(data) / batch_size

        for batch in dataloader:
            usr, jb, _, rt = batch
            # LOAD BATCH
            users = usr.to(device)
            jobs = jb.to(device)  # career
            # genders = g.to(device)
            ratings = rt.to(device)

            # PREDICTIONS
            y_hat = ncf(users.squeeze(1), jobs.squeeze(1))

            # BINARY CROSS-ENTROPY LOSS
            loss1 = loss(y_hat, ratings.float())

            predicted_probs = ncf(all_users, all_items)
            avg_epsilon = m.compute_edf(all_genders, predicted_probs, data.num_jobs, all_items, device)

            # criteroin hinge
            loss2 = torch.max(torch.tensor(0.0).to(device), (avg_epsilon - epsilonBase))

            final_loss = loss1 + fairness_thres*loss2

            optimizer.zero_grad()
            final_loss.backward()
            optimizer.step()

            if j % int(1 + it_per_epoch / 10) == 0:
                print(f"Progress: {round(100 * j / it_per_epoch)}%")
            j += 1
        ht, ndcg = evaluate_fine_tune(ncf, data.test, top_k, random_samples)
        print(f'Hit Ratio: {ht}  NDCG: {ndcg}   LOSS1: {loss1}  LOSS2: {loss2} ')

# -----------------------------------------------------------------

In [209]:
ht, ndcg = evaluate_fine_tune(ncf, data.test, top_k, random_samples)
# train_normal()
ht.mean()

KeyError: 'Column not found: job'

In [ ]:
'''MEASURE THE FAIRNESS OF THE MODEL'''
def fairness_measures(model,df_val,num_items):
    model.eval()
    users, items = torch.LongTensor(df_val.uid.to_numpy()).to(device), torch.LongTensor(df_val.job.to_numpy()).to(device)
    y_hat = model(users, items)

    avg_epsilon = m.compute_edf(all_genders.cpu(),y_hat,num_items,items,device)
    U_abs = m.compute_absolute_unfairness(all_genders.cpu(),y_hat,num_items,items,device)

    avg_epsilon = avg_epsilon.cpu().detach().numpy().reshape((-1,)).item()
    print(f"average differential fairness: {avg_epsilon: .3f}")

    U_abs = U_abs.cpu().detach().numpy().reshape((-1,)).item()
    print(f"absolute unfairness: {U_abs: .3f}")

fairness_measures(ncf, data.test, n_careers)

In [210]:
torch.save(ncf.state_dict(), "models/DF_NCF")

,uid,mid,rating
0,1,425,1
1,4,48,1
2,4,1337,1
3,7,313,1
4,8,490,1
...,...,...,...
9943,6038,1616,1
9944,6039,990,1
9945,6039,1572,1
9946,6039,3050,1
